## Show tables with Scala and Spark



In [1]:
spark.catalog.listTables.show(false)

+--------------------------------+--------+-----------+---------+-----------+
|name                            |database|description|tableType|isTemporary|
+--------------------------------+--------+-----------+---------+-----------+
|btl_departures_arrivals_airports|default |null       |EXTERNAL |false      |
|btl_distances                   |default |null       |EXTERNAL |false      |
|int_airports                    |default |null       |EXTERNAL |false      |
|int_departures                  |default |null       |EXTERNAL |false      |
+--------------------------------+--------+-----------+---------+-----------+



In [4]:
spark.sql("select * from default.btl_distances where estarrivalairport = 'EDDM'").show

+-------------------+-----------------+--------------+----------------+-----------------+-----------------+----------------+-----------------+------------------+---------------------+
|estdepartureairport|estarrivalairport|      arr_name|arr_latitude_deg|arr_longitude_deg|         dep_name|dep_latitude_deg|dep_longitude_deg|          distance|could_be_done_by_rail|
+-------------------+-----------------+--------------+----------------+-----------------+-----------------+----------------+-----------------+------------------+---------------------+
|               LSZB|             EDDM|Munich Airport|       48.353802|          11.7861|Bern-Belp Airport|       46.913419|         7.499747|358.82802268724186|                 true|
+-------------------+-----------------+--------------+----------------+-----------------+-----------------+----------------+-----------------+------------------+---------------------+



In [5]:
spark.table("default.int_departures").where($"estarrivalairport"==="LEPA").show

+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+---------+------+--------+---+--------------+
|arrivalAirportCandidatesCount|callsign|departureAirportCandidatesCount|estArrivalAirport|estArrivalAirportHorizDistance|estArrivalAirportVertDistance|estDepartureAirport|estDepartureAirportHorizDistance|estDepartureAirportVertDistance|firstSeen|icao24|lastSeen| dt|dl_ts_captured|
+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+---------+------+--------+---+--------------+
+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+---

## Select data by using DataObjects configured in SmartDataLake



In [7]:
// import smartdatalake
import io.smartdatalake.config.SdlConfigObject.stringToDataObjectId
import io.smartdatalake.config.ConfigToolbox
import io.smartdatalake.workflow.dataobject._
import io.smartdatalake.workflow.ActionPipelineContext
import io.smartdatalake.workflow.action.SDLExecutionId
import io.smartdatalake.app.SmartDataLakeBuilderConfig
import io.smartdatalake.workflow.ExecutionPhase
implicit val ss = spark // make Spark session available implicitly

In [8]:
// read config from mounted directory
val (registry, globalConfig) = ConfigToolbox.loadAndParseConfig(Seq("/mnt/config"), Some(this.getClass.getClassLoader))
// Create the context used by SDL objects
implicit val context = ConfigToolbox.getDefaultActionPipelineContext(spark, registry)

In [9]:
// get a dataobject
val dataIntAirports = registry.get[DeltaLakeTableDataObject]("int-airports")
val dataIntDepartures = registry.get[DeltaLakeTableDataObject]("int-departures")

In [10]:
dataIntDepartures.dropTable

In [11]:
dataIntDepartures.getSparkDataFrame().show

+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|arrivalAirportCandidatesCount|callsign|departureAirportCandidatesCount|estArrivalAirport|estArrivalAirportHorizDistance|estArrivalAirportVertDistance|estDepartureAirport|estDepartureAirportHorizDistance|estDepartureAirportVertDistance| firstSeen|icao24|  lastSeen|      dt|      dl_ts_captured|
+-----------------------------+--------+-------------------------------+-----------------+------------------------------+-----------------------------+-------------------+--------------------------------+-------------------------------+----------+------+----------+--------+--------------------+
|                            4|SSR07BM |                              0|             LIMC|                      